# common

In [ ]:
// // test

open testing

()



## types

In [ ]:
inl types () =
    rust.types ()
    sm'.types ()

()



## prototype

In [ ]:
prototype (~:>) r : forall t. t -> r

()



## run_target

In [ ]:
// // test

run_target function
    | Fsharp (Native) => fun () => $"1uy"
    | _ => fun () => $"2uy"
|> _assert_eq 1u8

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : uint8 option = None
    let v1 : bool = true in let mutable _v0 = v0
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : uint8 = 2uy
    v2
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v3 : uint8 = 2uy
    v3
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v4 : uint8 = 2uy
    v4
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v5 : uint8 = 1uy
    v5
#endif
    
#if FABLE_COMPILER_TYPESCRIPT
    let v6 : uint8 = 2uy
    v6
#endif
    
#if FABLE_COMPILER_PYTHON
    let v7 : uint8 = 2uy
    v7
#endif
    |> fun x -> _v0 <- Some x
    let v8 : uint8 = _v0.Value
    let v9 : string = $"%A{v8}"
    System.Console.WriteLine v9
    let v10 : bool = v8 = 1uy
    let v12 : bool =
        if v10 then
            true
        else
            method1(v10)
    let v13 : string = $"__expect / actual: %A{v8} / expected: %A{1uy}"
    let v14 : bool = v12 = false
    if v14 then
  

In [ ]:
// // test
// // rust=

run_target function
    | Rust (Native) => fun () => $"1uy"
    | _ => fun () => $"2uy"
|> _assert_eq 1u8

.rs output:
1




.fsx:
let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : uint8 option = None
    let v1 : bool = true in let mutable _v0 = v0
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v2 : uint8 = 1uy
    v2
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v3 : uint8 = 2uy
    v3
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v4 : uint8 = 2uy
    v4
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v5 : uint8 = 2uy
    v5
#endif
    
#if FABLE_COMPILER_TYPESCRIPT
    let v6 : uint8 = 2uy
    v6
#endif
    
#if FABLE_COMPILER_PYTHON
    let v7 : uint8 = 2uy
    v7
#endif
    |> fun x -> _v0 <- Some x
    let v8 : uint8 = _v0.Value
    let v9 : string = $"%A{v8}"
    System.Console.WriteLine v9
    let v10 : bool = v8 = 1uy
    let v12 : bool =
        if v10 then
            true
        else
            method1(v10)
    let v13 : string = $"__expect / actual: %A{v8} / expected: %A{1uy}"
    let v14 : bool = v12 = false
    if v14 t

## upcast

In [ ]:
inl upcast forall t u. (x : t) : u =
    $'!x :> `u '

()



## downcast

In [ ]:
inl downcast forall t u. (x : t) : u =
    $'!x :?> `u '

()



## pair

In [ ]:
// // test

pair 1i32 2i32
|> _assert_eq (1, 2)

let rec method0 () : unit =
    let v0 : string = $"%A{struct (1, 2)}"
    System.Console.WriteLine v0
    let v1 : string = $"__expect / actual: %A{struct (1, 2)} / expected: %A{struct (1, 2)}"
    ()
method0()

struct (1, 2)


## new_pair

In [ ]:
inl new_pair forall a b. (a : a) (b : b) : pair a b =
    $"!a, !b"

()



## from_pair

In [ ]:
inl from_pair forall a b. (pair : pair a b) : a * b =
    $"let (a, b) = !pair"
    $"a", $"b"

()



In [ ]:
// // test

new_pair "a" 1i32
|> from_pair
|> _assert_eq ("a", 1i32)

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = "a"
    let v1 : (string * int32) = v0, 1
    let (a, b) = v1
    let v2 : string = a
    let v3 : int32 = b
    let v4 : string = $"%A{struct (v2, v3)}"
    System.Console.WriteLine v4
    let v5 : bool = v2 = "a"
    let v7 : bool =
        if v5 then
            let v6 : bool = v3 = 1
            v6
        else
            false
    let v9 : bool =
        if v7 then
            true
        else
            method1(v7)
    let v10 : string = $"__expect / actual: %A{struct (v2, v3)} / expected: %A{struct (v0, 1)}"
    let v11 : bool = v9 = false
    if v11 then
        failwith<unit> v10
method0()

struct ("a", 1)


In [ ]:
// // test

new_pair "a" (new_pair 1i32 "b")
|> from_pair

(a, (1, b)) Item1 a Item2 (1, b) Item1 1 Item2 b

let rec method0 () : struct (string * (int32 * string)) =
    let v0 : string = "b"
    let v1 : (int32 * string) = 1, v0
    let v2 : string = "a"
    let v3 : (string * (int32 * string)) = v2, v1
    let (a, b) = v3
    let v4 : string = a
    let v5 : (int32 * string) = b
    struct (v4, v5)
method0()



## memoize

In [ ]:
nominal lazy t = $'Lazy<`t>'

inl memoize forall t. (fn : () -> t) : () -> t =
    inl fn = join fn
    inl result : lazy t = $'lazy !fn ()'
    fun () => $'!result.Value'

()



In [ ]:
// // test

inl count = mut 0i32
inl add =
    fun () => count <- *count + 1
    |> memoize

add ()
add ()
add ()

*count
|> _assert_eq 1

type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : (unit -> unit) = method1(v0)
    let v2 : Lazy<unit> = lazy v1 ()
    v2.Value
    v2.Value
    v2.Value
    let v3 : int32 = v0.l0
    let v4 : string = $"%A{v3}"
    System.Console.WriteLine v4
    let v5 : bool = v3 = 1
    let v7 : bool =
        if v5 then
            true
        else
            method2(v5)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{1}"
    let v9 : bool = v7 = false
    if v9 then
        failwith<unit> v8
method0()

1


## to_any

In [ ]:
inl to_any forall t. (obj : t) : any =
    $"!obj"

instance (~:>) any = to_any

()



## not'

In [ ]:
inl not' x =
    join not x

()



## (+.)

In [ ]:
inl (+.) forall t. (a : t) (b : t) : t =
    $"!a + !b"

()



## (-.)

In [ ]:
inl (-.) forall t. (a : t) (b : t) : t =
    $"!a - !b"

()



## (*.)

In [ ]:
inl (*.) forall t. (a : t) (b : t) : t =
    $"!a * !b"

()



## (/.)

In [ ]:
inl (/.) forall t. (a : t) (b : t) : t =
    $"!a / !b"

()



## (=.)

In [ ]:
inl (=.) forall t. (a : t) (b : t) : bool =
    $"!a = !b"

()



## (<>.)

In [ ]:
inl (<>.) forall t. (a : t) (b : t) : bool =
    $"!a <> !b"

()



## (||>)

In [ ]:
inl (||>) (arg1, arg2) fn =
    arg2 |> fn arg1

()



In [ ]:
// // test

(3i32, 2i32)
||> fun a b => a - b
|> _assert_eq 1

let rec method0 () : unit =
    let v0 : string = $"%A{1}"
    System.Console.WriteLine v0
    let v1 : string = $"__expect / actual: %A{1} / expected: %A{1}"
    ()
method0()

1


## flip

In [ ]:
inl flip fn a b =
    fn b a

()



In [ ]:
// // test

(1i32, 2i32)
||> flip pair
|> _assert_eq (2, 1)

let rec method0 () : unit =
    let v0 : string = $"%A{struct (2, 1)}"
    System.Console.WriteLine v0
    let v1 : string = $"__expect / actual: %A{struct (2, 1)} / expected: %A{struct (2, 1)}"
    ()
method0()

struct (2, 1)


## join_body

In [ ]:
inl join_body body acc x =
    if var_is x |> not
    then body acc x
    else
        inl acc = dyn acc
        join body acc x

()



In [ ]:
// // test

inl rec fold_list f s = function
    | Cons (x, x') => fold_list f (f s x) x'
    | Nil => s

()



In [ ]:
// // test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (+) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9 + v0
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : string = $"%A{v3}"
    System.Console.WriteLine v4
    let v5 : bool = v3 = 15
    let v7 : bool =
        if v5 then
            true
        else
            method2(v5)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{15}"
    let v9 : bool = v7 = false
    if v9 then
        failwith<unit> v8
method0()

15


In [ ]:
// // test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (join_body (+)) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32, v1 : int32) : int32 =
    let v2 : int32 = v1 + v0
    v2
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = 9
    let v2 : int32 = method2(v0, v1)
    let v3 : int32 = v2 + 2
    let v4 : int32 = v3 + 1
    let v5 : string = $"%A{v4}"
    System.Console.WriteLine v5
    let v6 : bool = v4 = 15
    let v8 : bool =
        if v6 then
            true
        else
            method3(v6)
    let v9 : string = $"__expect / actual: %A{v4} / expected: %A{15}"
    let v10 : bool = v8 = false
    if v10 then
        failwith<unit> v9
method0()

15


## join_body_unit

In [ ]:
inl join_body_unit body d x =
    if var_is d |> not
    then body x
    else
        inl x = dyn x
        join body x

()



In [ ]:
// // test

[ 5i32; 4; join 3; 2; 1 ]
|> fold_list (fun acc n => join_body_unit ((+) acc) n n) 0
|> _assert_eq 15

let rec method1 () : int32 =
    3
and method2 (v0 : int32) : int32 =
    let v1 : int32 = 9 + v0
    v1
and method3 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : int32 = method1()
    let v1 : int32 = method2(v0)
    let v2 : int32 = v1 + 2
    let v3 : int32 = v2 + 1
    let v4 : string = $"%A{v3}"
    System.Console.WriteLine v4
    let v5 : bool = v3 = 15
    let v7 : bool =
        if v5 then
            true
        else
            method3(v5)
    let v8 : string = $"__expect / actual: %A{v3} / expected: %A{15}"
    let v9 : bool = v7 = false
    if v9 then
        failwith<unit> v8
method0()

15


## nameof

In [ ]:
inl nameof x : string =
    $"nameof !x"

()



## get_environment_variable

In [ ]:
inl get_environment_variable (var : string) : string =
    $"System.Environment.GetEnvironmentVariable !var"

()



In [ ]:
// // test

fun () => failwith "test"
|> _throws
|> optionm.map sm'.format_exception
|> _assert_eq (Some "System.Exception: test")

type [<Struct>] US0 =
    | US0_0 of f0_0 : exn
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : string
    | US1_1
let rec closure0 () () : unit =
    failwith<unit> "test"
and closure1 () (v0 : exn) : US0 =
    US0_0(v0)
and method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : (unit -> unit) = closure0()
    let v1 : US0 = US0_1
    let v2 : (exn -> US0) = closure1()
    let v3 : US0 = try v0 (); v1 with ex -> ex |> v2
    let v17 : US1 =
        match v3 with
        | US0_1 -> (* None *)
            US1_1
        | US0_0(v4) -> (* Some *)
            let v5 : string option = None
            let v6 : bool = true in let mutable _v5 = v5
            
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
            let v7 : string = $"%A{v4}"
            v7
#endif
            
#if FABLE_COMPILER_RUST && WASM
            let v8 : string = $"%A{v4}"
            v8
#endif
            
#if FABLE_COMPILER_RUST && CONTRACT
            let v9 : string = $"%A{v4}"
         

## disposable

In [ ]:
nominal disposable t = $"System.IDisposable"

()



## dispose

In [ ]:
inl dispose (disposable : disposable _) : () =
    disposable |> $'_.Dispose()'

()



## new_disposable

In [ ]:
inl new_disposable (fn : () -> ()) : disposable _ =
    run_target function
        | Rust _ => fun () =>
            global "type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()"
            $'new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr !fn "$0()" )'
        | Fsharp _ => fun () =>
            inl fn = join fn
            $'{ new System.IDisposable with member _.Dispose () = !fn () }'
        | _ => fun () => null ()

()



In [ ]:
// // test

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : System.IDisposable option = None
    let v2 : bool = true in let mutable _v1 = v1
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v3 : (unit -> unit) = closure0(v0)
    let v4 : System.IDisposable = new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr v3 "$0()" )
    v4
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v5 : System.IDisposable = new Disposable (fun () -> Fable.Core.RustInterop.emitRustExpr v3 "$0()" )
    v5
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v6 : System.IDisposable = new Disposable (fun () -> Fable.Core.RustInte

In [ ]:
// // test
// // rust=
// // print_code=false

inl new_disposable_test = mut 0i32
new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
|> fun x => x : disposable ()
|> dispose
*new_disposable_test |> _assert_eq 1

11:07:39 debug   #1 new_arg / arg: Arg {
    id: "arg",
    help: None,
    long_help: None,
    action: None,
    value_parser: None,
    blacklist: [],
    settings: ArgFlags(
        0,
    ),
    overrides: [],
    groups: [],
    requires: [],
    r_ifs: [],
    r_unless: [],
    short: None,
    long: None,
    aliases: [],
    short_aliases: [],
    disp_ord: None,
    val_names: [],
    num_vals: None,
    val_delim: None,
    default_vals: [],
    default_vals_ifs: [],
    terminator: None,
    index: None,
    help_heading: None,
    value_hint: None,
    default_missing_vals: [],
}


In [ ]:
// // test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_task
|> async.await_task
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : System.Threading.Tasks.Task option = None
    let v2 : bool = true in let mutable _v1 = v1
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v3 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v3
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v4 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v4
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v5 : System.Threading.Tasks.Task = null |> unbox<System.Threading.Tasks.Task>
    v5
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v6

In [ ]:
// // test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> use
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 1

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : Async<unit> option = None
    let v2 : bool = true in let mutable _v1 = v1
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v3 : Async<unit> = null |> unbox<Async<unit>>
    v3
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v4 : Async<unit> = null |> unbox<Async<unit>>
    v4
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v5 : Async<unit> = null |> unbox<Async<unit>>
    v5
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v6 : Async<unit> option = None
    let mutable _v6 = v6
    async {
    let v7 : System.IDisposable option = None


In [ ]:
// // test

inl new_disposable_test = mut 0i32
fun () =>
    new_disposable fun () => new_disposable_test <- *new_disposable_test + 1
    |> fun x => x : disposable ()
    |> ignore
    |> return
|> async.new_async
|> async.run_synchronously
*new_disposable_test |> _assert_eq 0

type Disposable (f : unit -> unit) = interface System.IDisposable with member _.Dispose () = f ()
type Mut0 = {mutable l0 : int32}
let rec closure0 (v0 : Mut0) () : unit =
    let v1 : int32 = v0.l0
    let v2 : int32 = v1 + 1
    v0.l0 <- v2
    ()
and method1 (v0 : Mut0) : (unit -> unit) =
    closure0(v0)
and method2 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : Mut0 = {l0 = 0} : Mut0
    let v1 : Async<unit> option = None
    let v2 : bool = true in let mutable _v1 = v1
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v3 : Async<unit> = null |> unbox<Async<unit>>
    v3
#endif
    
#if FABLE_COMPILER_RUST && WASM
    let v4 : Async<unit> = null |> unbox<Async<unit>>
    v4
#endif
    
#if FABLE_COMPILER_RUST && CONTRACT
    let v5 : Async<unit> = null |> unbox<Async<unit>>
    v5
#endif
    
#if !FABLE_COMPILER && !WASM && !CONTRACT
    let v6 : Async<unit> option = None
    let mutable _v6 = v6
    async {
    let v7 : System.IDisposable option = None


## return

In [ ]:
inl return forall t. (x : t) : () =
    $'return !x '

inl return' forall t. (x : t) : t =
    $'return !x '

()



## sleep

In [ ]:
inl sleep (n : i32) : () =
    run_target function
        | Fsharp (Native) => fun () => $'System.Threading.Thread.Sleep' n
        | _ => fun () => ()

()



## retry_fn

In [ ]:
inl retry_fn forall t. retries (fn : () -> t) : option t =
    let rec loop retry =
        try
            fun () =>
                if retry < retries
                then fn () |> Some
                else None
            fun ex =>
                trace Warning
                    fun () => "retry_fn"
                    fun () =>
                        $'$"retry: {!retry} / ex: %A{!ex} / {!_locals ()}"'
                sleep 1
                None
        |> function
            | Some x => x
            | None => loop (retry + 1)
    loop 0

()



In [ ]:
// // test

inl retry_fn_test = mut 0i32
fun () =>
    retry_fn_test <- *retry_fn_test + 1
    *retry_fn_test
|> retry_fn 3i32
|> _assert_eq (Some 1i32)

module State = let mutable trace_state = None
type Mut0 = {mutable l0 : int32}
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1
and [<Struct>] US2 =
    | US2_0
    | US2_1
    | US2_2
    | US2_3
    | US2_4
and Mut1 = {mutable l0 : int64}
and Mut2 = {mutable l0 : bool}
and Mut3 = {mutable l0 : US2}
and Mut4 = {mutable l0 : (string -> unit)}
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64
    | US4_1
let rec closure0 (v0 : Mut0, v1 : int32) () : US0 =
    let v2 : bool = v1 < 3
    if v2 then
        let v3 : int32 = v0.l0
        let v4 : int32 = v3 + 1
        v0.l0 <- v4
        let v5 : int32 = v0.l0
        US0_0(v5)
    else
        US0_1
and closure1 () (v0 : US0) : US1 =
    US1_0(v0)
and closure3 () () : string =
    let v0 : string = "retry_fn"
    v0
and closure5 () () : string =
    let v0 : string = ""
    v0
and closure4 (v0 : int32, v1 : e

In [ ]:
// // test

inl retry_fn_test = mut 0i32
fun () =>
    if *retry_fn_test >= 2
    then *retry_fn_test
    else
        retry_fn_test <- *retry_fn_test + 1
        failwith "test"
|> retry_fn 3i32
|> _assert_eq (Some 2i32)

module State = let mutable trace_state = None
type Mut0 = {mutable l0 : int32}
and [<Struct>] US0 =
    | US0_0 of f0_0 : int32
    | US0_1
and [<Struct>] US1 =
    | US1_0 of f0_0 : US0
    | US1_1
and [<Struct>] US2 =
    | US2_0
    | US2_1
    | US2_2
    | US2_3
    | US2_4
and Mut1 = {mutable l0 : int64}
and Mut2 = {mutable l0 : bool}
and Mut3 = {mutable l0 : US2}
and Mut4 = {mutable l0 : (string -> unit)}
and [<Struct>] US3 =
    | US3_0 of f0_0 : int64
    | US3_1
and [<Struct>] US4 =
    | US4_0 of f0_0 : int64
    | US4_1
let rec closure0 (v0 : Mut0, v1 : int32) () : US0 =
    let v2 : bool = v1 < 3
    if v2 then
        let v3 : int32 = v0.l0
        let v4 : bool = v3 >= 2
        let v9 : int32 =
            if v4 then
                let v5 : int32 = v0.l0
                v5
            else
                let v6 : int32 = v0.l0
                let v7 : int32 = v6 + 1
                v0.l0 <- v7
                failwith<int32> "test"
        US0_0(v9)
    else
        U

## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    inl new_disposable x : _ () = new_disposable x
    $"let new_disposable x = !new_disposable x" : ()

    inl retry_fn (r : i32) (x : () -> _) : optionm'.option' () = retry_fn r x |> optionm'.box
    $"let retry_fn x = !retry_fn x" : ()
    inl memoize (fn : () -> ()) : () -> () = memoize fn
    $"let memoize x = !memoize x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func0<$0>")>]
#endif
type Func0<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Func1<$0, $1>")>]
#endif
type Func0<'T, 'U> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Box<$0>")>]
#endif
type Box<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("dyn $0")>]
#endif
type Dyn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("$0 + Send")>]
#endif
type Send<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn() -> $0")>]
#endif
type Fn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn()")>]
#endif
type FnUnit = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("FnOnce() -> $0")>]
#endif
type FnOnce<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0)")>]
#endif
type ActionFn<'T> = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("Fn($0, $1)")>]